# ntopconf 2023 - takeaway #1
### Documentation https://www.ntop.org/guides/ntopng/api/python/index.html


### Prerequisites
```shell
1) Running ntopng 
2) apt-get install python3 python3-pip git # python3, python3-pip and git installed
3) git clone https://github.com/pmorphin/ntop-2023.git
4) cd ntop-2023/
5) pip3 install -r requirements.txt 
```

### Run ntopng live mode
```shell
ap@nt:~/ntopng/$./ntopng --dont-change-user
```

### Run ntopng to analyze a pcap file
```shell
ap@nt:~/ntopng/$./ntopng --dont-change-user -i path/to/the/traffic/file.pcapng
```

### Login and list all available interfaces

In [ ]:
import os

from ntopng.ntopng import Ntopng

username     = "admin"
password     = "admin1"
ntopng_url   = "http://localhost:3000"

try:
	my_ntopng = Ntopng(username, password, None, ntopng_url)
	interfaces_available = my_ntopng.get_interfaces_list() # Return all available interfaces
	for interface in interfaces_available:
		print (interface)
except ValueError as e:
	print(e)
	os._exit(-1)

![Alt text](img/ntop-dns-traffic.png)

In [ ]:
import os
import json

from ntopng.ntopng import Ntopng

username     = "admin"
password     = "admin1"
ntopng_url   = "http://localhost:3000"

try:
	my_ntopng = Ntopng(username, password, None, ntopng_url)
	interfaces_available = my_ntopng.get_interfaces_list()
	#for interface in interfaces_available:
	#	print (interface)

	# {'ifid': 1, 'name': 'ens33', 'ifname': 'ens33'}
	# {'ifid': 2, 'name': 'lo', 'ifname': 'lo'}
	# OR
	# {'ifname': 'dns_tunnelling.pcapng', 'name': 'dns_tunnelling.pcapng', 'ifid': 3}

	my_interface = my_ntopng.get_interface(3)

	# get_active_flows_paginated(currentPage, perPage)
	traffic = my_interface.get_active_flows_paginated(1,500)
	traffic_b = json.dumps(traffic, indent=4)
	print (traffic_b)

except ValueError as e:
	print(e)
	os._exit(-1)

![Alt text](img/ntop-dns-traffic-info.png)

### Extend get_active_flows_paginated () results with "info" field 
```shell
ap@nt:~/$cat ntopng/python/ntopng/interface.py
```
```python
...
  def get_active_flows_paginated(self, currentPage, perPage):
        """
        Retrieve the (paginated) list of active flows for the specified interface
        
        :param currentPage: The current page
        :type currentPage: int
        :param perPage: The number of results per page
        :type perPage: int
        :return: All active flows
        :rtype: array
        """
        return(self.ntopng_obj.request(self.rest_v2_url + "/get/flow/active.lua", {"ifid": self.ifid, "currentPage": currentPage, "perPage": perPage}))
...
``` 

``` shell
ap@nt:~/$cat ntopng/scripts/lua/rest/v2/get/flow/active.lua
```
``` lua
...
for _key, value in ipairs(flows_stats) do
   local record = {}

   local key = value["ntopng.key"]

   record["key"] = string.format("%u", value["ntopng.key"])
   record["hash_id"] = string.format("%u", value["hash_entry_id"])
   -- training
   record["info"] = value["info"]

   record["first_seen"] = value["seen.first"]
   record["last_seen"] = value["seen.last"]

   local client = {}

   local cli_name = flowinfo2hostname(value, "cli")
   client["name"] = stripVlan(cli_name)
   client["ip"] = value["cli.ip"]
   client["port"] = value["cli.port"]
...
```

In [ ]:
import os
import json

from ntopng.ntopng import Ntopng

username     = "admin"
password     = "admin1"
ntopng_url   = "http://localhost:3000"

try:
	my_ntopng = Ntopng(username, password, None, ntopng_url)
	interfaces_available = my_ntopng.get_interfaces_list()
	#for interface in interfaces_available:
	#	print (interface)

	# {'ifid': 1, 'name': 'ens33', 'ifname': 'ens33'}
	# {'ifid': 2, 'name': 'lo', 'ifname': 'lo'}
	# OR
	# {'ifname': 'dns_tunnelling.pcapng', 'name': 'dns_tunnelling.pcapng', 'ifid': 3}

	my_interface = my_ntopng.get_interface(3)

	# get_active_flows_paginated(currentPage, perPage)
	traffic = my_interface.get_active_flows_paginated(1,500)
	traffic_b = json.dumps(traffic, indent=4)
	print (traffic_b)
except ValueError as e:
	print(e)
	os._exit(-1)